In [28]:
from nltk import word_tokenize, sent_tokenize
from nltk.corpus import stopwords
import nltk
import os
from collections import defaultdict, Counter
from sets import Set
import sklearn
from sklearn import svm
from sklearn import linear_model
import numpy as np
import scipy as sci
import timeit
import sys

from nltk.stem import WordNetLemmatizer


from math import pow
from numpy import linalg as LA
from math import fabs
from scipy import sparse
import numpy, sys
import numpy, gzip, sys
from numpy.linalg import norm
from cPickle import load, dump
rng = numpy.random.RandomState(1234)

In [29]:
class LR():
    def __init__(self, alpha,lmbda,maxiter):
        self.alpha = float(alpha) # learning rate for gradient ascent
        self.lmbda = float(lmbda) # regularization constant
        self.epsilon = 0.00001 # convergence measure
        self.maxiter = int(maxiter) # the maximum number of iterations through the data before stopping
        self.threshold = 0.5 # the class prediction threshold

  

    def sigmoid(self,x):
        x= 1. / (1. + np.exp(-x))
        return x

    def getDataFiles(self,files,train, vocab , vocab_no):
    
        rows = []
        cols = []
        data = []
        pol = []
        review_sum = count = tp = fp = fn = 0
        folds = ['neg','pos']
       # path = "../../Movies/edu-input-final/"
       
    #     files = [os.path.join(path, fname) for fname in os.listdir(path) if fname.endswith('.edus')]
        for filename in files:
                # fname = (filename.split(".edus"))[0]
           
                f = open(filename, 'r')
                content = f.read()
                content = content.decode('utf-8').encode('ascii','ignore')
                content = content.replace('<br />','').lower()
                words = word_tokenize(content)
                words2 = [w for w in words if not w in stopwords.words('english')]
                cnt = Counter(words2)
                for word in cnt:
                    if word not in vocab and train:    
                        vocab[word] = vocab_no[0]
                        vocab_no[0] = vocab_no[0] + 1
                    if word in vocab:
                        rows.append(count)
                        cols.append(vocab[word])
                        data.append(cnt[word])
                

                fname = str(filename.split("/")[-1])
        
                rating = int(fname.split(".")[0].split("_")[1])
                if rating <6:
                    pol.append(-1)
                if rating >6 :
                    pol.append(1)
                    
                count = count + 1
      
        mat = sci.sparse.csr_matrix((data,(rows,cols)), shape=(count,vocab_no[0])).todense()
        return mat, pol


    def fit(self, X, y):
        """
        This function optimizes the parameters for the logistic regression classification model from training 
        data using learning rate alpha and regularization constant lmbda
        @post: parameter(theta) optimized by gradient descent
        """
        # X = self.add_ones(X_) # prepend ones to training set for theta_0 calculations
        
        # initialize optimization arrays
        self.n = X.shape[1] # the number of features
        self.m = X.shape[0] # the number of instances
        self.probas = np.zeros(self.m, dtype='float') # stores probabilities generated by the logistic function
        self.theta = np.zeros(self.n, dtype='float') # stores the model theta

        # iterate through the data at most maxiter times, updating the theta for each feature
        # also stop iterating if error is less than epsilon (convergence tolerance constant)
        print "iter | magnitude of the gradient"
        for iteration in xrange(self.maxiter):
            # calc probabilities
            self.probas = self.get_proba(X)
            print iteration
            # calculate the gradient and update theta
            
            gw = np.zeros(self.n, dtype='float')
            if y<0:
                y=0
            k = (1.0/self.m) * np.dot(X.T,(self.probas - y).T)
            # print k.shape[0]
            for i in range(k.shape[0]):
                # print k[i,0]
                gw[i]=k[i,0] 
        
            gw =gw + (self.lmbda * self.theta)/self.m  # regularize using the lmbda term
           
            self.theta -= self.alpha * gw # update parameters
            
            # calculate the magnitude of the gradient and check for convergence
            loss = np.linalg.norm(gw)
            if self.epsilon > loss:
                break
            
            print iteration, ":", loss

    def get_proba(self, X):
        return 1.0 / (1 + np.exp(- np.dot(X, self.theta)))

    def predict_proba(self, X):
        """
        Returns the set of classification probabilities based on the model theta.
        @parameters: X - array-like of shape = [n_samples, n_features]
                     The input samples.
        @returns: y_pred - list of shape = [n_samples]
                  The probabilities that the class label for each instance is 1 to standard output.
        """
        # X_ = self.add_ones(X)
        return self.get_proba(X)

    def predict(self, X):
        """
        Classifies a set of data instances X based on the set of trained feature theta.
        @parameters: X - array-like of shape = [n_samples, n_features]
                     The input samples.
        @returns: y_pred - list of shape = [n_samples]
                  The predicted class label for each instance.
        """
        y_pred =[]
        pred =  self.predict_proba(X)
       
        for i in range(pred.shape[1]):
            proba =pred[0,i] 
            if proba >self.threshold:
                y_pred.append(1)
            else:
                y_pred.append(-1)
        # y_pred = [proba > self.threshold for proba in self.predict_proba(X)]
        return y_pred

    def add_ones(self, X):
        # prepend a column of 1's to dataset X to enable theta_0 calculations
        # print X.shape[0]
        return np.hstack((np.zeros(shape=(X.shape[0],1), dtype='float') + 1, X))

    def sigmoid_grad(self,f):
        sig = sigmoid(f)
        return sig * (1 - sig)
   

def compute_accuracy(y_test, y_pred):
        """
        @returns: The precision of the classifier, (correct labels / instance count)
        """
        correct = 0
        for i in range(len(y_test)):
            if y_pred[i] == y_test[i]:
                correct += 1
        return float(correct) / len(y_test)
    
def savemodel(fname,D):
        """ Save model into fname
        """
        if not fname.endswith('.pickle.gz'):
            fname = fname + '.pickle.gz'
        # D = self.getparams()
        with gzip.open(fname, 'w') as fout:
            dump(D, fout)
        print 'Save model into file {}'.format(fname)


def loadmodel( fname):
        """ Load model from fname
        """
        with gzip.open(fname, 'r') as fin:
            D = load(fin)
        return D
        print 'Load model from file: {}'.format(fname)       


 
  

In [32]:
D =loadmodel("data.pickle.gz")
train_mat,train_pol  = D["tdata"] ,D["tpos"]
test_mat,test_pol = D["vdata"] , D["vpos"]


In [34]:
# My Logistic Regression
alpha=1
lmbda=.1 
maxiter=1000
prob_train =[]
for i in range(len(train_pol)):
        if train_pol[i] < 0:
            prob_train.append(0)
        else:
            prob_train.append(1)
lr = LR(alpha,lmbda,maxiter)
lr.fit(train_mat,prob_train)
y_pred = lr.predict(test_mat)

print compute_accuracy(test_pol,y_pred)
   
    # train_mat,train_pol = lr.getDataFiles(tfiles,True,vocab,vocab_no)
    # test_mat,test_pol = lr.getDataFiles(vfiles,False,vocab,vocab_no)
    # D = {"tdata":train_mat, "tpos":train_pol, "vdata":test_mat,"vpos":test_pol}
    # savemodel("data.pickle.gz",D)


iter | magnitude of the gradient
0
0 : 0.502138741894
1
1 : 6.05131365232
2
2 : 10.113468762
3
3 : 9.91251476402
4
4 : 10.1134526364
5
5 : 9.91252901079
6
6 : 10.1134358227
7
7 : 9.91254102813
8
8 : 10.1134115458
9
9 : 9.91255188783
10
10 : 10.1131084279
11
11 : 9.91256210532
12
12 : 10.1115228384
13
13 : 9.91257180335
14
14 : 10.1112421203
15
15 : 9.91258061569
16
16 : 10.1105896155
17
17 : 9.91258422442
18
18 : 10.0795459275
19
19 : 9.91254501971
20
20 : 10.0540862962
21
21 : 9.9121707536
22
22 : 9.91259304032
23
23 : 9.90899676608
24
24 : 9.83808427934
25
25 : 9.89347322112
26
26 : 9.7634072027
27
27 : 9.86829027146
28
28 : 9.73031756293
29
29 : 9.85147785794
30
30 : 9.70915577423
31
31 : 9.82783256825
32
32 : 9.69118713034
33
33 : 9.77529254658
34
34 : 9.66511398393
35
35 : 9.71311563688
36
36 : 9.62061517514
37
37 : 9.65900419045
38
38 : 9.59238665652
39
39 : 9.62389757864
40
40 : 9.53760571039
41
41 : 9.52864860513
42
42 : 9.51321919734
43
43 : 9.51835397222
44
44 : 9.50110024706

/System/Library/Frameworks/Python.framework/Versions/2.7/lib/python2.7/site-packages/IPython/kernel/__main__.py:104: RuntimeWarning: overflow encountered in exp


In [21]:
# Scikit Code
lr_sci = linear_model.LogisticRegression(penalty='l2', dual=False, tol=0.0001, C=1.0, fit_intercept=True, 
                                     intercept_scaling=1, class_weight=None, random_state=None)
lr_sci.fit(train_mat,train_pol)
y2 = lr_sci.predict(test_mat)
compute_accuracy(test_pol,y2)
# print test_pol
# print y2
# print y_pred
# print sklearn.metrics.f1_score(y2, test_pol)
# sklearn.metrics.accuracy_score(y2, test_pol)

0.8088235294117647